# 데이터 로딩

In [3]:
import pandas as pd
from difflib import SequenceMatcher

# 데이터 불러오기
product = pd.read_csv('../dataset/products_no_china_no_token_v2.csv')
w_product = product[product['judge'] == 0][['CATALOG_DESC']]
prohibited_words = pd.read_csv('../dataset/IPR리스트_231218.csv')

# 금지어리스트와의 단어 유사성
- 입력되는 문장을 단어별로 분할하여 하나씩 금지어 리스트와 유사성 비교
- 비슷한 단어, 금지어, 유사도, 카테고리, 금지어코드, 문장 컬럼으로 데이터프레임 생성
  - 비슷한 단어: 입력된 데이터에서 금지어리스트와 유사한 단어
  - 금지어: 금지어 단어와 유사한 금지어데이터의 단어
  - 유사도: 비슷한단어와 금지어단어의 유사도
  - 카테고리: 금지어리스트의 사유
  - 금지어코드: 금지어리스트의 코드
  - 문장: 비슷한 단어 앞의 한단어, 뒤의 한단어 포함된 문장

In [10]:
def find_similar_word_with_context(sentence, prohibited_words):
    similar_words = []
    
    # 문장을 단어로 분할
    words_sentence = sentence.split()
    
    for i, word in enumerate(words_sentence):
        # 앞 뒤 한 단어씩 추가한 문장 생성
        context_start = max(0, i - 1)
        context_end = min(i + 2, len(words_sentence))
        context_words = words_sentence[context_start:context_end]
        context_sentence = ' '.join(context_words)
        
        # 각 단어와 금지어 리스트의 단어들을 비교하여 유사성 측정
        for index, row in prohibited_words.iterrows():
            prohibited_word = row['KEYWORD']
            prohibited_reason = row['사유']
            prohibited_code = row['코드']  # 가정: 'IPR리스트_231218.csv' 파일에 '코드' 컬럼이 있다고 가정
            matcher = SequenceMatcher(None, word, prohibited_word)
            similarity_ratio = matcher.ratio()
            
            # 임계값을 설정하여 유사도가 일정 수준 이상인 단어들을 유사한 단어로 처리
            if similarity_ratio > 0.7:  # 임계값은 상황에 따라 조정 가능
                similar_words.append({
                    '비슷한 단어': word,
                    '금지어': prohibited_word,
                    '유사도': similarity_ratio,
                    '문장': context_sentence,
                    '카테고리': prohibited_reason,
                    '금지어 코드': prohibited_code
                })
    
    return similar_words

# 금지어 리스트 단어 추가
- 금지어리스트와 유사한 단어를 단어리스트 목록에 추가할건지 단어별로 사용자에게 물어보고 사용자가 1을 입력하면 해당 단어를 금지어 리스트에 저장

In [11]:
# 비교할 문장 정의
sentence = w_product['CATALOG_DESC'][0]

# 유사한 단어와 문장 찾기
similar_words = find_similar_word_with_context(sentence, prohibited_words)

if similar_words:
    similar_words_df = pd.DataFrame(similar_words)
    display(similar_words_df)
    
    unique_similar_words = similar_words_df['비슷한 단어'].unique()
    for word in unique_similar_words:
        save_to_prohibited = input(f"단어 '{word}'를 금지어 리스트에 추가하시겠습니까? (1: 추가, 0: 건너뛰기): ")
        if save_to_prohibited == '1':
            word_info = similar_words_df[similar_words_df['비슷한 단어'] == word].iloc[0]
            new_row = {
                'KEYWORD': word,
                '사유': word_info['카테고리'],
                '코드': word_info['금지어 코드']
            }
            prohibited_words = pd.concat([prohibited_words, pd.DataFrame([new_row])], ignore_index=True)
            print(f"단어 '{word}'가 금지어 리스트에 추가되었습니다.")
        else:
            print(f"단어 '{word}'는 추가되지 않았습니다.")
    
    # TODO: 필요한 작업(예: prohibited_words를 파일로 저장)을 수행합니다.
else:
    print("비슷한 단어를 찾지 못했습니다.")

,비슷한 단어,금지어,유사도,문장,카테고리,금지어 코드
0,phenylamino,phenethylamine,0.800000,fluoro phenylamino piperidine1,금지약품 (Prohibited Drugs),2
1,phenylamino,phenylamino,1.000000,fluoro phenylamino piperidine1,금지약품 (Prohibited Drugs),2
2,piperidine1,Piperidine,0.857143,phenylamino piperidine1 2,금지약품 (Prohibited Drugs),2
3,dimethylethyl,methylfentanyl,0.740741,1 dimethylethyl ester,금지약품 (Prohibited Drugs),2
4,dimethylethyl,dimethylone,0.750000,1 dimethylethyl ester,금지약품 (Prohibited Drugs),2
5,phenylamino,phenethylamine,0.800000,fluoro phenylamino piperidine1,금지약품 (Prohibited Drugs),2
6,phenylamino,phenylamino,1.000000,fluoro phenylamino piperidine1,금지약품 (Prohibited Drugs),2
7,piperidine1,Piperidine,0.857143,phenylamino piperidine1 piperidinecarboxylic,금지약품 (Prohibited Drugs),2
8,fluorophenyl,fluorofentanyl,0.769231,4 fluorophenyl amino,금지약품 (Prohibited Drugs),2
9,dimethylethyl,methylfentanyl,0.740741,1 dimethylethyl ester1,금지약품 (Prohibited Drugs),2


단어 'phenylamino'를 금지어 리스트에 추가하시겠습니까? (1: 추가, 0: 건너뛰기):  1


단어 'phenylamino'가 금지어 리스트에 추가되었습니다.


단어 'piperidine1'를 금지어 리스트에 추가하시겠습니까? (1: 추가, 0: 건너뛰기):  0


단어 'piperidine1'는 추가되지 않았습니다.


...

단어 'data'는 추가되지 않았습니다.


단어 'charles'를 금지어 리스트에 추가하시겠습니까? (1: 추가, 0: 건너뛰기):  0


단어 'charles'는 추가되지 않았습니다.


단어 'rose'를 금지어 리스트에 추가하시겠습니까? (1: 추가, 0: 건너뛰기):  0


단어 'rose'는 추가되지 않았습니다.


In [12]:
prohibited_words.tail()

,KEYWORD,사유,코드
956,dimethylone,금지약품 (Prohibited Drugs),2
957,fluorobutyrfentanyl,금지약품 (Prohibited Drugs),2
958,fub amb,금지약품 (Prohibited Drugs),2
959,phenylamino,금지약품 (Prohibited Drugs),2
960,phenylamino,금지약품 (Prohibited Drugs),2
